In [1]:
import os
import copy
import time
import asyncio
import warnings
import logging

from pathlib import Path

import numpy as np
from astropy.io import fits

from scipy import ndimage
from scipy.signal import medfilt
from scipy.ndimage.filters import gaussian_filter
from astropy.modeling import models, fitting


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [7, 6]

#Import CWFS package
from lsst import cwfs
from lsst.cwfs.instrument import Instrument
from lsst.cwfs.algorithm import Algorithm
from lsst.cwfs.image import Image, readFile, aperture2image, showProjection
import lsst.cwfs.plots as plots

import wget
from lsst.ts import salobj

%matplotlib inline

In [2]:
cam = salobj.Remote(salobj.Domain(), "GenericCamera", 1)

In [3]:
await cam.start_task

In [4]:
zern = []
dz_vec = [0.5, 1.0, 1.5]

In [5]:
for i in [1,3,6]:
    print("================================================================================")
    print("Waiting intra image")
    cam.evt_endReadout.flush()
    intra_endron = await cam.evt_endReadout.next(flush=False)
    if "intra" not in intra_endron.imageName:
        print(f"Got {intra_endron.imageName}. Not an intra image... skipping")
        continue
    print(intra_endron.imageName)

    extra_endron = await cam.evt_endReadout.next(flush=False, timeout=120)
    print(extra_endron.imageName)

    intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
    extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
    download_done = False
    while not download_done:
        try:
            intra_fname = wget.download(intra_wget_url)
        except Exception as e:
            print(f"Error downloading {intra_wget_url}. waiting 5s and trying again.")
            await asyncio.sleep(5.)
        else:
            download_done = True
    print(f"============")
    print(f'{intra_fname}')

    download_done = False
    while not download_done:
        try:
            extra_fname = wget.download(extra_wget_url)
        except Exception as e:
            print(f"Error downloading {intra_wget_url}. waiting 5s and trying again.")
            await asyncio.sleep(5.)
        else:
            download_done = True
            
    print(f'{extra_fname}')
    print(f"============")

    # Open fits files, note you can start here if your files exist already

    #intra_fname='I1_intra_20190908.fits'
    intra_exp=(fits.open(intra_fname))[0].data

    #extra_fname='I2_extra_20190908.fits'
    extra_exp=(fits.open(extra_fname))[0].data

    # Show images added together to estimate centroid
    plt.imshow(intra_exp+extra_exp)

    # Define the image it will use to perform final center
    im_shape=intra_exp.shape
    side=300
    # Set to False to declare centroid manually!
    if True:
        im= (intra_exp+extra_exp)
        im_filtered = medfilt(im,[5,5])
        im_filtered -= int(np.median(im_filtered))
        mean = np.mean(im_filtered)
    #     im_filtered[im_filtered < mean] = 0.
    #     im_filtered[im_filtered > mean] = 1.
        # iter 1
        ceny, cenx = np.array(ndimage.measurements.center_of_mass(im_filtered), dtype=int)
        # iter 2
        intra_square = intra_exp[ceny-side:ceny+side, cenx-side:cenx+side] 
        extra_square = extra_exp[ceny-side:ceny+side, cenx-side:cenx+side]
        im= (intra_square+extra_square)
        im_filtered = medfilt(im,[5,5])
        im_filtered -= int(np.median(im_filtered))
        mean = np.mean(im_filtered)
        im_filtered[im_filtered < mean] = 0.
        im_filtered[im_filtered > mean] = 1.
        # iter 1
        cy2, cx2 = np.array(ndimage.measurements.center_of_mass(im_filtered), dtype=int)
        print(cy2-side, cx2-side)    
        ceny += (cy2-side)
        cenx += (cx2-side)
    else:
        # Manually declare where the center is
        ceny=350 #round(im_shape[0]/2)
        cenx=450 # round(im_shape[1]/2)
    side=150 # side length of image
    print('Creating stamps of centroid [y,x] = [{},{}] with a side length of {} pixels'.format(ceny,cenx,side))
    im_shape=intra_exp.shape
    intra_square = intra_exp[ceny-side:ceny+side, cenx-side:cenx+side] 
    extra_square = extra_exp[ceny-side:ceny+side, cenx-side:cenx+side]

    #Create object for CWFS code
    # Select where your object is, but ours will be on-axis
    fieldXY = [0.0,0.0]

    # I1/I2 get modified down below, so reset here
    I1 = None; I2=None
    I1 = Image(intra_square, fieldXY, Image.INTRA)
    I2 = Image(extra_square, fieldXY, Image.EXTRA)

    # Declare instrument
    inst=Instrument(f'auxtel_{i+1}',I1.sizeinPix) # example
    inst.offset = dz_vec[i]*0.041
    hex_to_focus_scale = 41.0
    offset=0.5 *hex_to_focus_scale # [mm] multiply hexapod dz by magnification factor
    pixelsize = 3.6e-6*4 # 4 is the binning
    # FIXME: put an assertion here and calculate binning above based on change in image size, also pull offset from filename!

    print('Offset should be :{} [mm] at the focus, {} [mm] at the hexapod'.format(offset, offset/hex_to_focus_scale))
    print('Offset in file is :{} [mm] at the focus'.format(1e3*inst.offset))
    print('pixelSize should be: {}'.format(pixelsize))

    #declare algorithm
    # declare algorithm - exponential solver.
    algo=Algorithm('exp',inst,1) # example - but only want 11 Zernikes (Num_of_Zernikes)


    # Plot images
    plots.plotImage(I1.image,'intra') 
    #plots.plotImage(I_focus.image,'focus')
    plots.plotImage(I2.image,'extra')
    print('Extra-focal filename: {}'.format(extra_fname))
    print('Intra-focal filename: {}'.format(intra_fname))

    algo.runIt(inst,I1,I2,'onAxis')
    plots.plotImage(I1.image,'intra', mask=algo.pMask) 
    #plots.plotImage(I_focus.image,'focus')
    plots.plotImage(I2.image,'extra', mask=algo.pMask)    
    print(algo.zer4UpNm)
    plots.plotZer(algo.zer4UpNm[0:9],'nm')
    zern.append(algo.zer4UpNm[0:9])

Waiting intra image
1579928616-alpCMa_intra-0-1
1579928660-alpCMa_extra-0-1
1579928616-alpCMa_intra-0-1.fits
1579928660-alpCMa_extra-0-1.fits


/opt/lsst/software/stack/python/miniconda3-4.7.10/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/lsst/software/stack/python/miniconda3-4.7.10/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: cannot convert float NaN to integer